In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import geopandas as gpd
from fiona.crs import from_epsg
import geopandas as gpd
import os
import pandas as pd
import re

In [3]:
# 1. 나눔 폰트 다운
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [4]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [5]:
import chardet

In [10]:
df4 = pd.read_csv('/content/drive/MyDrive/new_project1/data/chodata/서울시 안심귀갓길 서비스.csv', encoding= 'CP949')

In [11]:
df4

,시군구명,읍면동 코드,읍면동명,관리기관,전화번호,경도,위도,세부위치,안심귀갓길 명
0,서울특별시 종로구,1111016400,종로6가,혜화경찰서,02-762-4400,127.007590,37.573204,서울특별시 종로구 율곡로 271,혜화안심06
1,서울특별시 종로구,1111016400,종로6가,혜화경찰서,02-762-4400,127.004970,37.572605,서울특별시 종로구 종로39길 40,혜화안심06
2,서울특별시 종로구,1111016400,종로6가,혜화경찰서,02-762-4400,127.006540,37.571434,서울특별시 종로구 종로41길 3,혜화안심06
3,서울특별시 중구,1114016500,황학동,중부(광희)지구대,02-2233-1444,127.019800,37.568450,서울특별시 중구 황학동 마장로 89,중부안심08
4,서울특별시 중구,1114016200,신당동,중부(약수)지구대,02-2234-8112,127.012850,37.559240,서울특별시 중구 다산로 175(신당동),중부안심05
...,...,...,...,...,...,...,...,...,...
342,서울특별시 강남구,1168010800,논현동,강남경찰서,02-557-7000,127.042680,37.510155,서울특별시 강남구 봉은사로 327,강남안심18
343,서울특별시 강남구,1168010700,신사동,강남경찰서,02-557-7000,127.028990,37.526314,서울특별시 강남구 압구정로30길 18,강남안심17
344,서울특별시 강남구,1168010700,신사동,강남경찰서,02-557-7000,127.030106,37.527206,서울특별시 강남구 논현로176길 20,강남안심17
345,서울특별시 강남구,1168010700,신사동,강남경찰서,02-557-7000,127.024430,37.520520,서울특별시 강남구 논현로153길 46(신사동 555-13),강남안심15


In [12]:
df5 = df4[['시군구명', '읍면동 코드', '읍면동명']]

In [13]:
df5['시군구명'] = df5['시군구명'].str.replace('서울특별시 ', '')

<ipython-input-13-5ca02fcf55cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['시군구명'] = df5['시군구명'].str.replace('서울특별시 ', '')


In [14]:
df5 = df5.rename(columns={'시군구명': '자치구', '읍면동 코드': 'code10', '읍면동명': '행정동명'})

In [15]:
df5

,자치구,code10,행정동명
0,종로구,1111016400,종로6가
1,종로구,1111016400,종로6가
2,종로구,1111016400,종로6가
3,중구,1114016500,황학동
4,중구,1114016200,신당동
...,...,...,...
342,강남구,1168010800,논현동
343,강남구,1168010700,신사동
344,강남구,1168010700,신사동
345,강남구,1168010700,신사동


In [16]:
df5 = df5[['자치구', '행정동명', 'code10']]

In [19]:
df5.rename(columns={'자치구': '시군구', '행정동명': '읍면동'}, inplace=True)

In [20]:
df5.drop(columns='code10', inplace=True)

In [21]:
df5

,시군구,읍면동
0,종로구,종로6가
1,종로구,종로6가
2,종로구,종로6가
3,중구,황학동
4,중구,신당동
...,...,...
342,강남구,논현동
343,강남구,신사동
344,강남구,신사동
345,강남구,신사동


In [22]:
filtered_data = df5[df5['읍면동'].str.contains('\.') | df5['읍면동'].str.contains('·')]

In [23]:
df5 = df5.groupby(['시군구', '읍면동']).size().reset_index(name='안심귀갓길 개수')

In [24]:
df5

,시군구,읍면동,안심귀갓길 개수
0,강남구,논현동,4
1,강남구,대치동,2
2,강남구,삼성동,3
3,강남구,신사동,3
4,강남구,역삼동,8
...,...,...,...
125,중구,필동3가,1
126,중구,황학동,1
127,중구,회현동3가,1
128,중랑구,면목동,5


In [25]:
df5['읍면동'] = df5['읍면동'].apply(lambda x: re.sub(r'[·.]', '', x))
df5[df5['읍면동'].str.contains('[·.]')]

,시군구,읍면동,안심귀갓길 개수


In [26]:
dong_shp = gpd.read_file('/content/drive/MyDrive/new_project1/data/seoul_shp/dong/bnd_dong_11_2023_2023_2Q.shp')
dong_shp.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1..."
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1..."
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1..."
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1..."
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1..."


In [29]:
df5['구동'] = df5['시군구'] + ' ' +df5['읍면동']
df5.head()

,시군구,읍면동,안심귀갓길 개수,구동
0,강남구,논현동,4,강남구 논현동
1,강남구,대치동,2,강남구 대치동
2,강남구,삼성동,3,강남구 삼성동
3,강남구,신사동,3,강남구 신사동
4,강남구,역삼동,8,강남구 역삼동


In [32]:
df5['안심귀갓길 개수'] = df5['안심귀갓길 개수'].apply(lambda x: 0 if x =='-' else x)
df5['안심귀갓길 개수'] = df5['안심귀갓길 개수'].apply(lambda x: int(x))
df5['안심귀갓길 개수'].unique()

<ipython-input-32-391ca8fed817>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['안심귀갓길 개수'] = df5['안심귀갓길 개수'].apply(lambda x: 0 if x =='-' else x)
<ipython-input-32-391ca8fed817>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['안심귀갓길 개수'] = df5['안심귀갓길 개수'].apply(lambda x: int(x))


array([ 4,  2,  3,  8,  1,  7, 12, 14, 17,  5,  9,  6])

In [ ]:
df5 = df5[['구동', '안심귀갓길 개수']]
df5

In [34]:
dong_shp['ADM_CD'] = dong_shp['ADM_CD'].astype('O')

In [58]:
dong_shp[dong_shp['ADM_NM'] == '길동']

,BASE_DATE,ADM_NM,ADM_CD,geometry
416,20230701,길동,11250740,"POLYGON ((968944.218 1949625.263, 968943.832 1..."


In [59]:
df5[df5['구동'] == '강동구 길동 ']

,구동,안심귀갓길 개수


In [37]:
region_code = pd.read_csv('/content/drive/MyDrive/new_project1/data/complete/region_code.csv')
region_code.head()

,ADM_CD,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동
0,11010530,서울특별시 종로구 사직동,1101053,1111053000,11010530,11110530,11110530,서울특별시,종로구,사직동
1,11010540,서울특별시 종로구 삼청동,1101054,1111054000,11010540,11110540,11110540,서울특별시,종로구,삼청동
2,11010550,서울특별시 종로구 부암동,1101055,1111055000,11010550,11110550,11110550,서울특별시,종로구,부암동
3,11010560,서울특별시 종로구 평창동,1101056,1111056000,11010560,11110560,11110560,서울특별시,종로구,평창동
4,11030740,서울특별시 용산구 한남동,1103074,1117068500,11030740,11170685,11170685,서울특별시,용산구,한남동


In [38]:
dong_shp['ADM_CD'] = dong_shp['ADM_CD'].astype('int')

In [39]:
region_code['ADM_CD'] = region_code['ADM_CD'].astype('int')

In [40]:
map_base = dong_shp.merge(region_code,
                          how = 'left',
                          on = 'ADM_CD')
map_base.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053,1111053000,11010530,11110530,11110530,서울특별시,종로구,사직동
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054,1111054000,11010540,11110540,11110540,서울특별시,종로구,삼청동
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055,1111055000,11010550,11110550,11110550,서울특별시,종로구,부암동
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056,1111056000,11010560,11110560,11110560,서울특별시,종로구,평창동
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074,1117068500,11030740,11170685,11170685,서울특별시,용산구,한남동


In [41]:
map_base['ADM_NM'] = map_base['ADM_NM'].apply(lambda x: re.sub(r'\·', '', x))

In [42]:
map_base['key1'] = map_base['시군구'] + ' ' + map_base['ADM_NM']
map_base['key2'] = map_base['시군구'] + ' ' + map_base['읍면동']

In [43]:
map_base[['key1', 'key2']]

,key1,key2
0,종로구 사직동,종로구 사직동
1,종로구 삼청동,종로구 삼청동
2,종로구 부암동,종로구 부암동
3,종로구 평창동,종로구 평창동
4,용산구 한남동,용산구 한남동
...,...,...
421,종로구 가회동,종로구 가회동
422,종로구 종로1234가동,종로구 종로1234가동
423,서초구 방배2동,서초구 방배2동
424,강북구 미아동,강북구 미아동


In [46]:
final3 = map_base.merge(df5.rename(columns = {'구동' : 'key1'}),
                       how = 'left',
                       on = 'key1')
final3

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동,key1,key2,안심귀갓길 개수
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053,1111053000,11010530,11110530,11110530,서울특별시,종로구,사직동,종로구 사직동,종로구 사직동,NaN
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054,1111054000,11010540,11110540,11110540,서울특별시,종로구,삼청동,종로구 삼청동,종로구 삼청동,NaN
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055,1111055000,11010550,11110550,11110550,서울특별시,종로구,부암동,종로구 부암동,종로구 부암동,NaN
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056,1111056000,11010560,11110560,11110560,서울특별시,종로구,평창동,종로구 평창동,종로구 평창동,1.0
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074,1117068500,11030740,11170685,11170685,서울특별시,용산구,한남동,용산구 한남동,용산구 한남동,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,20230701,가회동,11010600,"POLYGON ((954894.795 1954614.580, 954888.290 1...",서울특별시 종로구 가회동,1101060,1111060000,11010600,11110600,11110600,서울특별시,종로구,가회동,종로구 가회동,종로구 가회동,NaN
422,20230701,종로1234가동,11010610,"POLYGON ((954918.389 1954371.538, 954926.411 1...",서울특별시 종로구 종로1·2·3·4가동,1101061,1111061500,11010610,11110615,11110615,서울특별시,종로구,종로1234가동,종로구 종로1234가동,종로구 종로1234가동,NaN
423,20230701,방배2동,11220630,"POLYGON ((954967.724 1942969.970, 954985.115 1...",서울특별시 서초구 방배2동,1122063,1165061000,11220630,11650610,11650610,서울특별시,서초구,방배2동,서초구 방배2동,서초구 방배2동,NaN
424,20230701,미아동,11090700,"POLYGON ((958301.054 1959603.612, 958292.609 1...",서울특별시 강북구 미아동,1109070,1130553500,11090700,11305535,11305535,서울특별시,강북구,미아동,강북구 미아동,강북구 미아동,4.0


In [47]:
final3[final3['안심귀갓길 개수'].isna()]

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동,key1,key2,안심귀갓길 개수
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053,1111053000,11010530,11110530,11110530,서울특별시,종로구,사직동,종로구 사직동,종로구 사직동,NaN
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054,1111054000,11010540,11110540,11110540,서울특별시,종로구,삼청동,종로구 삼청동,종로구 삼청동,NaN
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055,1111055000,11010550,11110550,11110550,서울특별시,종로구,부암동,종로구 부암동,종로구 부암동,NaN
5,20230701,왕십리2동,11040520,"POLYGON ((958262.940 1951830.159, 958266.662 1...",서울특별시 성동구 왕십리2동,1104052,1120052000,11040520,11200520,11200520,서울특별시,성동구,왕십리제2동,성동구 왕십리2동,성동구 왕십리제2동,NaN
6,20230701,마장동,11040540,"POLYGON ((959182.984 1952644.101, 959192.863 1...",서울특별시 성동구 마장동,1104054,1120054000,11040540,11200540,11200540,서울특별시,성동구,마장동,성동구 마장동,성동구 마장동,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,20230701,교남동,11010580,"POLYGON ((952572.048 1953258.829, 952573.174 1...",서울특별시 종로구 교남동,1101058,1111058000,11010580,11110580,11110580,서울특별시,종로구,교남동,종로구 교남동,종로구 교남동,NaN
421,20230701,가회동,11010600,"POLYGON ((954894.795 1954614.580, 954888.290 1...",서울특별시 종로구 가회동,1101060,1111060000,11010600,11110600,11110600,서울특별시,종로구,가회동,종로구 가회동,종로구 가회동,NaN
422,20230701,종로1234가동,11010610,"POLYGON ((954918.389 1954371.538, 954926.411 1...",서울특별시 종로구 종로1·2·3·4가동,1101061,1111061500,11010610,11110615,11110615,서울특별시,종로구,종로1234가동,종로구 종로1234가동,종로구 종로1234가동,NaN
423,20230701,방배2동,11220630,"POLYGON ((954967.724 1942969.970, 954985.115 1...",서울특별시 서초구 방배2동,1122063,1165061000,11220630,11650610,11650610,서울특별시,서초구,방배2동,서초구 방배2동,서초구 방배2동,NaN


In [48]:
final3['안심귀갓길 개수'] = final3['안심귀갓길 개수'].fillna(0)

In [49]:
final3[final3['안심귀갓길 개수'].isna()]

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,시도명,시군구,읍면동,key1,key2,안심귀갓길 개수


In [50]:
final3.to_csv('/content/drive/MyDrive/new_project1/data/complete/cho3.csv', index=False, encoding='utf-8-sig')